In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
import numpy as np


data = pd.read_csv("./combine_data_2000-2018.csv")

print(data.shape)


(6218, 16)


In [4]:
# Limit to only a specific position, WR to start
# data = data[data['Pos'] == 'CB']

# print(data.shape)

In [5]:
# Remove rows that do not have a "Round"

print(data.shape)

(6218, 16)


In [6]:
features = ["Pos", "Ht", "Wt", "Forty", "Vertical", "BenchReps", "BroadJump", "Cone", "Shuttle", "Year"]

X, y = data[features], data['Round']

print(X.shape)

(6218, 10)


In [7]:
# Update all nan to be the mean

# for f in features:
# 	if (f == 'Pos'):
# 		continue

# 	X[f].fillna((X[f].mean()))


X = X.fillna(0)
y = y.fillna(8)

X

,Pos,Ht,Wt,Forty,Vertical,BenchReps,BroadJump,Cone,Shuttle,Year
0,OLB,76,252,4.55,0.0,0.0,0.0,0.00,0.00,2000
1,RB,72,218,4.58,0.0,0.0,0.0,0.00,0.00,2000
2,OT,76,334,5.56,25.0,23.0,94.0,8.48,4.98,2000
3,TE,74,253,4.97,29.0,0.0,104.0,7.29,4.49,2000
4,CB,74,206,4.55,34.0,0.0,123.0,7.18,4.15,2000
...,...,...,...,...,...,...,...,...,...,...
6213,ILB,74,238,4.86,29.5,15.0,0.0,0.00,0.00,2018
6214,G,75,313,0.00,0.0,0.0,0.0,0.00,0.00,2018
6215,CB,73,190,4.52,0.0,8.0,120.0,0.00,4.18,2018
6216,ILB,73,236,4.60,36.0,23.0,117.0,7.38,4.48,2018


In [8]:
# Encode Position

X.Pos=X.Pos.astype('category').cat.codes

print(X.head())

   Pos  Ht   Wt  Forty  Vertical  BenchReps  BroadJump  Cone  Shuttle  Year
0   16  76  252   4.55       0.0        0.0        0.0  0.00     0.00  2000
1   20  72  218   4.58       0.0        0.0        0.0  0.00     0.00  2000
2   17  76  334   5.56      25.0       23.0       94.0  8.48     4.98  2000
3   23  74  253   4.97      29.0        0.0      104.0  7.29     4.49  2000
4    1  74  206   4.55      34.0        0.0      123.0  7.18     4.15  2000


In [9]:
# MinMaxScaler

feature_scaler = MinMaxScaler()
X = feature_scaler.fit_transform(X)

In [10]:
def train_predict(X_train, y_train, X_test, y_test):
	model =  RandomForestClassifier()
	model.fit(X_train, y_train)

	# 6 Evaluation: Test the model via predicting
	y_pred = model.predict(X_train)
	acc_train = accuracy_score(y_train, y_pred)
	print("accuracy score (training data): %.2f" % acc_train)
	y_pred = model.predict(X_test)
	acc_test = accuracy_score(y_test, y_pred)
	print("accuracy_score (test data): %.2f" % acc_test)

In [11]:
# Split the data into Train and Test so we can predict
TEST_PROP, RANDOM_SEED = 0.5, 0
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_PROP, random_state=RANDOM_SEED)

In [12]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)

print(kf)

KFold(n_splits=2, random_state=None, shuffle=False)

model =  RandomForestClassifier()

predictions = cross_val_predict(model, X, y, cv=kf)
scores = cross_val_score(model, X, y, cv=kf)

train_predict(X_train, y_train, X_test, y_test)
for index, score in np.ndenumerate(scores):
	print('k-Fold:', index[0], ':', score)

KFold(n_splits=10, random_state=None, shuffle=False)
accuracy score (training data): 1.00
accuracy_score (test data): 0.40
k-Fold: 0 : 0.3987138263665595
k-Fold: 1 : 0.39067524115755625
k-Fold: 2 : 0.3778135048231511
k-Fold: 3 : 0.3633440514469453
k-Fold: 4 : 0.3633440514469453
k-Fold: 5 : 0.3504823151125402
k-Fold: 6 : 0.3167202572347267
k-Fold: 7 : 0.32315112540192925
k-Fold: 8 : 0.3446054750402576
k-Fold: 9 : 0.5217391304347826
